In [1]:
from dataset import *
from constants import *
import datetime
import tensorflow as tf
from model import *

In [2]:
train_data, validation_data, test_data = get_data_generators(DATA_DIR, BATCH_SIZE, IMAGE_SIZE)

Found 83484 images belonging to 4 classes.
Found 32 images belonging to 4 classes.
Found 968 images belonging to 4 classes.


In [ ]:
model = resnet_model(input_shape=INPUT_SHAPE, num_classes = len(train_data.class_indices))

In [ ]:
model.summary()

In [ ]:
learning_rate = 1e-3
model.compile(
  #optimizer=tf.keras.optimizers.SGD(lr=0.005, momentum=0.9), 
  optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate), 
  loss=tf.keras.losses.CategoricalCrossentropy(),
  metrics=['accuracy'])

In [ ]:
log_path = LOG_DIR + 'resnet50v2_retrain_8layers_lr_1e-3' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_path, histogram_freq=1)

In [ ]:
checkpoint_kwargs = dict(monitor='loss', verbose=1, save_best_only=True)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(MODEL_CHECKPOINT_DIR+'resnet50v2_retrain_8layers_lr_1e-3.hdf5', **checkpoint_kwargs)

In [ ]:
steps_per_epoch = train_data.samples // train_data.batch_size
validation_steps = validation_data.samples // validation_data.batch_size

fit_kwargs = dict(steps_per_epoch=steps_per_epoch, epochs=10, validation_steps=validation_steps, 
                  validation_data=validation_data, verbose=1, callbacks = [tensorboard_callback, model_checkpoint])

In [ ]:
out = model.fit(train_data, **fit_kwargs)

In [4]:
model = tf.keras.models.load_model(MODEL_CHECKPOINT_DIR+'resnet50v2_retrain_8layers_lr_1e-3.hdf5')

In [5]:
loss, acc = model.evaluate(validation_data, verbose=2)

1/1 - 0s - loss: 0.5052 - accuracy: 0.7812


In [6]:
loss, acc = model.evaluate(test_data, verbose=2)

31/31 - 2s - loss: 0.4859 - accuracy: 0.8089
